In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Melanoma"
cohort = "GSE215868"

# Input paths
in_trait_dir = "../../input/GEO/Melanoma"
in_cohort_dir = "../../input/GEO/Melanoma/GSE215868"

# Output paths
out_data_file = "../../output/preprocess/Melanoma/GSE215868.csv"
out_gene_data_file = "../../output/preprocess/Melanoma/gene_data/GSE215868.csv"
out_clinical_data_file = "../../output/preprocess/Melanoma/clinical_data/GSE215868.csv"
json_path = "../../output/preprocess/Melanoma/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I've analyzed the issue and will provide a corrected version that properly structures the sample characteristics for the `geo_select_clinical_features` function.

```python
import pandas as pd
import numpy as np
import os
import re
from typing import Optional, Dict, Any, Callable

# 1. Gene Expression Data Availability
# Based on series title and summary, this appears to be gene expression data
is_gene_available = True

# 2.1 Data Availability
# For trait: We're looking for melanoma indicator. "long-term benefit" in row 4
trait_row = 4  # "long-term benefit" represents melanoma therapy outcome
# For age: Available in row 0
age_row = 0
# For gender: Not available in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    if pd.isna(value):
        return None
    # Extract the value after the colon
    match = re.search(r'long-term benefit:\s*(.*)', value)
    if match:
        # Convert to binary: YES=1, NO=0
        trait_value = match.group(1).strip()
        if trait_value == "YES":
            return 1
        elif trait_value == "NO":
            return 0
    return None

def convert_age(value):
    if pd.isna(value):
        return None
    # Extract age value after the colon
    match = re.search(r'age:\s*(\d+)', value)
    if match:
        return int(match.group(1))
    return None

def convert_gender(value):
    # Function defined but not used as gender data is not available
    return None

# 3. Save Metadata
# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=trait_row is not None
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Since we don't have direct access to clinical_data.csv, we need to create a DataFrame
    # that matches the expected structure for geo_select_clinical_features
    
    # Create a DataFrame where each row is a feature type and each column will be a sample
    # This matches the expected structure for geo_select_clinical_features
    
    # Sample characteristics dictionary from the previous step
    sample_chars = {
        0: ['age: 71', 'age: 59', 'age: 63', 'age: 44', 'age: 56', 'age: 58', 'age: 74', 'age: 51', 'age: 76', 'age: 16', 'age: 72', 'age: 73', 'age: 34', 'age: 64', 'age: 68', 'age: 36', 'age: 52', 'age: 54', 'age: 85', 'age: 82', 'age: 46', 'age: 69', 'age: 40', 'age: 75', 'age: 86', 'age: 50', 'age: 77', 'age: 88', 'age: 38', 'age: 61'],
        1: ['bor: CR', 'bor: SD', 'bor: PR', 'bor: PD', 'bor: UK'],
        2: ['pfs_days: 270', 'pfs_days: 588', 'pfs_days: 308', 'pfs_days: 942', 'pfs_days: 85', 'pfs_days: 351', 'pfs_days: 81', 'pfs_days: 932', 'pfs_days: 979', 'pfs_days: 685', 'pfs_days: 204', 'pfs_days: 16', 'pfs_days: 80', 'pfs_days: 652', 'pfs_days: 407', 'pfs_days: 1887', 'pfs_days: 36', 'pfs_days: 141', 'pfs_days: 153', 'pfs_days: 341', 'pfs_days: 1102', 'pfs_days: 96', 'pfs_days: 916', 'pfs_days: 62', 'pfs_days: 761', 'pfs_days: 136', 'pfs_days: 217', 'pfs_days: 102', 'pfs_days: 1552', 'pfs_days: 61'],
        3: ['pfs_index: 0', 'pfs_index: 1'],
        4: ['os_days: 350', 'long-term benefit: NO', 'long-term benefit: YES', 'os_days: 688', 'os_days: 377', 'os_days: 1833', 'os_days: 574', 'os_days: 446', 'os_days: 474', 'os_days: 415', 'os_days: 528', 'os_days: 605', 'os_days: 449', 'os_days: 259', 'os_days: 291', 'os_days: 376', 'os_days: 266', 'os_days: 123', 'os_days: 28', 'os_days: 301', 'os_days: 669'],
        5: ['os_index: 0', 'os_days: 1048', 'os_days: 911', 'os_days: 1354', 'os_days: 377', 'os_days: 514', 'os_days: 446', 'os_days: 959', 'os_days: 980', 'os_days: 418', 'os_days: 309', 'os_days: 678', 'os_days: 1010', 'os_days: 636', 'os_days: 1890', 'os_days: 238', 'os_days: 240', 'os_days: 681', 'os_days: 1169', 'os_days: 1037', 'os_days: 917', 'os_days: 200', 'os_days: 772', 'os_days: 160', 'os_days: 510', 'os_days: 1555', 'os_days: 128', 'os_days: 353', 'os_days: 229', 'os_days: 105'],
        6: ['itx: PEMBRO', 'os_index: 0', 'os_index: 1', 'itx: IPI+NIVO', 'itx: NIVO'],
        7: ['prior checkpoint blockade: NO', 'itx: PEMBRO', 'itx: NIVO', 'itx: IPI+NIVO', 'prior checkpoint blockade: YES', 'itx: NIVO+EXPERIMENTAL', 'itx: Pembro'],
        8: [np.nan, 'prior checkpoint blockade: YES', 'prior checkpoint blockade: NO']
    }
    
    # This appears to be a dictionary where keys are row indices and values are lists of characteristics
    # geo_select_clinical_features is expecting a DataFrame where rows are these characteristics
    clinical_data = pd.DataFrame(index=sample_chars.keys())
    
    # Extract clinical features using geo_select_clinical_features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(clinical_features)
    print("Clinical Features Preview:")
    print(preview)
    
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file),


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I understand the issue now - we need to handle the gzipped series matrix file properly. I'll implement the correct approach to extract the clinical features:

```python
import os
import pandas as pd
import json
import numpy as np
import gzip
import re
from typing import Optional, Callable, Dict, Any

# First, let's check what files are available in the cohort directory
print(f"Checking files in {in_cohort_dir}")
available_files = os.listdir(in_cohort_dir)
print(f"Available files: {available_files}")

# Load the series matrix file which contains sample characteristics
matrix_files = [f for f in available_files if "series_matrix" in f.lower()]
if matrix_files:
    matrix_file_path = os.path.join(in_cohort_dir, matrix_files[0])
    print(f"Loading series matrix file: {matrix_file_path}")
    
    # Read the gzipped matrix file
    with gzip.open(matrix_file_path, 'rt') as file:
        lines = file.readlines()
    
    # Extract sample characteristics and organize by type
    char_types = {}
    current_samples = []
    
    for line in lines:
        if line.startswith("!Sample_characteristics_ch1"):
            parts = line.strip().split('\t')
            if len(parts) > 1:
                # Extract the characteristic value for each sample
                char_value = parts[1]
                if ":" in char_value:
                    char_type, value = char_value.split(":", 1)
                    char_type = char_type.strip()
                    if char_type not in char_types:
                        char_types[char_type] = []
                    char_types[char_type].append(value.strip())
                else:
                    # Handle case where there's no colon separator
                    if "uncategorized" not in char_types:
                        char_types["uncategorized"] = []
                    char_types["uncategorized"].append(char_value)
        
        # Get sample identifiers
        elif line.startswith("!Sample_geo_accession"):
            parts = line.strip().split('\t')
            if len(parts) > 1:
                current_samples = parts[1:]
    
    # Print discovered characteristic types
    print("Discovered characteristic types:")
    for char_type, values in char_types.items():
        unique_values = set(values)
        print(f"Type: {char_type}")
        print(f"Unique values: {unique_values}")
        print(f"Count: {len(values)}")
        print()
    
    # Create a DataFrame for characteristics
    clinical_data = pd.DataFrame(index=current_samples)
    for char_type, values in char_types.items():
        if len(values) == len(current_samples):
            clinical_data[char_type] = values
    
    # Transpose for the format expected by geo_select_clinical_features
    clinical_data = clinical_data.T.reset_index()
    clinical_data.columns = ['characteristic'] + list(clinical_data.columns[1:])
    
    print("Clinical data preview:")
    print(clinical_data.head())
    
    # Determine feature availability based on the data
    # Let's examine characteristics to find trait, age and gender information
    is_gene_available = True  # Assuming this dataset contains gene expression data
    
    # These will be set based on examining the actual data
    trait_row = None
    age_row = None
    gender_row = None
    
    # Look for trait-related information (melanoma)
    for i, char_type in enumerate(clinical_data['characteristic']):
        char_type_lower = str(char_type).lower()
        # Search for trait information
        if 'disease' in char_type_lower or 'diagnosis' in char_type_lower or 'melanoma' in char_type_lower or 'tumor' in char_type_lower or 'cancer' in char_type_lower or 'status' in char_type_lower:
            # Check if there are at least two unique values (not constant)
            unique_vals = clinical_data.iloc[i, 1:].nunique()
            if unique_vals > 1:
                trait_row = i
                print(f"Found trait information in row {i}: {char_type}")
                print(f"Unique values: {clinical_data.iloc[i, 1:].unique()}")
        
        # Search for age information
        elif 'age' in char_type_lower:
            # Check if there are multiple unique values
            unique_vals = clinical_data.iloc[i, 1:].nunique()
            if unique_vals > 1:
                age_row = i
                print(f"Found age information in row {i}: {char_type}")
                print(f"Unique values: {clinical_data.iloc[i, 1:].unique()[:5]}")  # Show first 5 unique values
        
        # Search for gender information
        elif 'gender' in char_type_lower or 'sex' in char_type_lower:
            # Check if there are at least two unique values
            unique_vals = clinical_data.iloc[i, 1:].nunique()
            if unique_vals > 1:
                gender_row = i
                print(f"Found gender information in row {i}: {char_type}")
                print(f"Unique values: {clinical_data.iloc[i, 1:].unique()}")
    
    # Define conversion functions
    def convert_trait(value):
        if pd.isna(value) or value is None:
            return None
        value = str(value).strip().lower()
        
        # Convert to binary based on melanoma context
        if any(term in value for term in ['melanoma', 'tumor', 'cancer', 'malignant']):
            return 1
        elif any(term in value for term in ['normal', 'control', 'healthy', 'benign']):
            return 0
        else:
            return None
    
    def convert_age(value):
        if pd.isna(value) or value is None:
            return None
        value = str(value).strip()
        
        # Extract numeric age
        try:
            # Find numbers in the string
            age_match = re.search(r'(\d+)', value)
            if age_match:
                return float(age_match.group(1))
            else:
                return None
        except:
            return None
    
    def convert_gender(value):
        if pd.isna(value) or value is None:
            return None
        value = str(value).strip().lower()
        
        # Convert gender to binary (female=0, male=1)
        if any(term in value for term in ['female', 'f']):
            return 0
        elif any(term in value for term in ['male', 'm']):
            return 1
        else:
            return None
    
    # Determine if trait data is available
    is_trait_available = trait_row is not None
    
    # 3. Save Metadata - Initial filtering
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )
    
    # 4. Clinical Feature Extraction (if trait data is available)
    if trait_row is not None:
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age if age_row is not None else None,
            gender_row=gender_row,
            convert_gender=convert_gender if gender_row is not None else None
        )
        
        # Preview the results
        print("Selected clinical features preview:")
        preview = preview_df(selected_clinical_df)
        print(preview)
        
        # Save to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_


### Step 4: Dataset Analysis and Clinical Feature Extraction

In [ ]:
```python
import pandas as pd
import os
import json
import numpy as np

# List all files in the cohort directory to understand the available data
print("Files in the cohort directory:")
for file in os.listdir(in_cohort_dir):
    print(f"- {file}")

# Try to find the gene expression matrix file
expression_file = None
for file in os.listdir(in_cohort_dir):
    if any(term in file.lower() for term in ["expression", "matrix", "counts", "fpkm", "tpm", "series_matrix"]):
        if not any(excluded in file.lower() for excluded in ["mirna", "methyl"]):
            expression_file = os.path.join(in_cohort_dir, file)
            print(f"Found potential gene expression file: {file}")
            break

# Check for gene expression data availability
is_gene_available = expression_file is not None
print(f"Gene expression data available: {is_gene_available}")

# Try to load the series matrix file which should contain sample characteristics
clinical_data = None
if expression_file and os.path.exists(expression_file):
    try:
        # Series matrix files often contain both sample characteristics and expression data
        with open(expression_file, 'r') as f:
            lines = []
            i = 0
            # Read the first few lines to identify the structure
            for line in f:
                lines.append(line)
                i += 1
                if i > 100:  # Read first 100 lines to get header info
                    break
        
        # Check for sample characteristics section in these lines
        characteristic_lines = [line for line in lines if "!Sample_characteristics" in line]
        print(f"Found {len(characteristic_lines)} sample characteristic lines")
        
        if characteristic_lines:
            # Extract sample characteristics
            sample_chars = []
            for line in characteristic_lines:
                parts = line.strip().split('\t')
                sample_chars.append(parts[1:])  # Skip the !Sample_characteristics tag
            
            # Transpose the data to have samples as columns
            if sample_chars:
                sample_chars_transposed = list(map(list, zip(*sample_chars)))
                clinical_data = pd.DataFrame(sample_chars_transposed)
                
                # Find the row containing sample IDs if available
                id_row = None
                for i, row in clinical_data.iterrows():
                    if any(id_term in str(val).lower() for val in row for id_term in ["geo_accession", "sample", "id"]):
                        id_row = i
                        break
                
                if id_row is not None:
                    clinical_data.columns = clinical_data.iloc[id_row]
                
                print("Clinical data shape:", clinical_data.shape)
                print("Sample of clinical data:")
                print(clinical_data.head())
                
                # Examine unique values in each row to identify potential trait, age, and gender rows
                print("\nUnique values in each row:")
                for i, row in clinical_data.iterrows():
                    unique_vals = set(str(val) for val in row if not pd.isna(val))
                    if 1 < len(unique_vals) < 10:  # Show rows with 2-9 unique values
                        print(f"Row {i}: {unique_vals}")
    except Exception as e:
        print(f"Error processing series matrix file: {e}")

# If we couldn't extract clinical data from the series matrix, try to find a separate clinical file
if clinical_data is None:
    print("Looking for separate clinical data file...")
    for file in os.listdir(in_cohort_dir):
        if any(term in file.lower() for term in ["clinical", "pheno", "metadata", "phenotype", "sample"]):
            clinical_file = os.path.join(in_cohort_dir, file)
            try:
                clinical_data = pd.read_csv(clinical_file, sep='\t')
                print(f"Loaded clinical data from {file}")
                print("Clinical data shape:", clinical_data.shape)
                print("Sample of clinical data:")
                print(clinical_data.head())
                break
            except Exception as e:
                print(f"Error loading clinical file {file}: {e}")

# Based on our examination of the data, we'll set these variables
trait_row = None
age_row = None
gender_row = None

# If we have clinical data, analyze it to find trait, age, and gender information
if clinical_data is not None:
    # Check each row for trait, age, and gender information
    for i, row in clinical_data.iterrows():
        row_values = [str(val).lower() for val in row if not pd.isna(val)]
        row_text = ' '.join(row_values)
        
        # Check for melanoma/disease information
        if any(term in row_text for term in ["melanoma", "tumor", "cancer", "diagnosis", "disease", "status", "patient", "primary"]):
            unique_vals = set(row_values)
            if len(unique_vals) > 1:  # Ensure there's variation in the values
                print(f"Potential trait row {i}: {unique_vals}")
                trait_row = i
        
        # Check for age information
        if any(term in row_text for term in ["age", "years", "yr", "old"]):
            # Verify variety in ages
            if len(set(row_values)) > 1:
                print(f"Potential age row {i}: {set(row_values)}")
                age_row = i
        
        # Check for gender information
        if any(term in row_text for term in ["gender", "sex", "male", "female"]):
            # Verify variety in genders
            if len(set(row_values)) > 1:
                print(f"Potential gender row {i}: {set(row_values)}")
                gender_row = i

# Define conversion functions
def convert_trait(value):
    if value is None or pd.isna(value):
        return None
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 1 for melanoma, 0 for control
    value_lower = str(value).lower()
    if any(term in value_lower for term in ["melanoma", "tumor", "cancer", "case", "patient", "primary", "yes", "positive"]):
        return 1
    elif any(term in value_lower for term in ["control", "normal", "healthy", "non-melanoma", "nevus", "benign", "no", "negative"]):
        return 0
    return None

def convert_age(value):
    if value is None or pd.isna(value):
        return None
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Extract numeric age
    if isinstance(value, (int, float)):
        return float(value)
    
    import re
    # Try to extract numbers from the string
    numbers = re.findall(r'\d+', str(value))
    if numbers:
        return float(numbers[0])
    return None

def convert_gender(value):
    if value is None or pd.isna(value):
        return None
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 0 for female, 1 for male
    value_lower = str(value).lower()
    if any(term in value_lower for term in ["female", "f", "woman", "girl"]):
        return 0
    elif any(term in value_lower for term in ["male", "m", "man", "boy"]):
        return 1
    return None

# Determine trait availability
is_trait_available = trait_row is not None
print(f"Trait data available: {is_trait_available}")
print(f"Age data available: {age_row is not None}")
print(f"Gender data available: {gender_row is


### Step 5: Gene Data Extraction

In [ ]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


### Step 6: Gene Identifier Review

In [ ]:
# Looking at the gene identifiers in the gene expression data
# These appear to be human gene symbols (like A2M, ABCF1, ACVR1C, etc.)
# rather than probe IDs or other identifiers that would require mapping

requires_gene_mapping = False


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Get the paths to the SOFT and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
gene_data = get_genetic_data(matrix_file)
print(f"Extracted gene data with shape: {gene_data.shape}")

# Normalize gene symbols in the gene expression data
print(f"Top 10 gene indices before normalization: {gene_data.index[:10].tolist()}")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Top 10 gene indices after normalization: {normalized_gene_data.index[:10].tolist()}")
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")

# Create directory for gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# Define conversion functions for clinical data extraction
def convert_trait(value):
    if pd.isna(value):
        return None
    # Extract the value after 'long-term benefit:' if present
    match = re.search(r'long-term benefit:\s*(.*)', str(value))
    if match:
        value = match.group(1).strip()
        if value.upper() == "YES":
            return 1
        elif value.upper() == "NO":
            return 0
    return None

def convert_age(value):
    if pd.isna(value):
        return None
    # Extract age value after the colon
    match = re.search(r'age:\s*(\d+)', str(value))
    if match:
        return int(match.group(1))
    return None

def convert_gender(value):
    # Not used as gender data isn't available
    return None

# 2. Extract clinical features using the clinical data from step 1
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Based on the sample characteristics dictionary from Step 1, 
# long-term benefit (trait) is in row 4
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=4,  # "long-term benefit" information is in row 4
    convert_trait=convert_trait,
    age_row=0,   # Age information is in row 0
    convert_age=convert_age,
    gender_row=None,  # No gender information available
    convert_gender=None
)

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Saved clinical data to {out_clinical_data_file}")

# 3. Link clinical and genetic data
try:
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Shape of linked data: {linked_data.shape}")
    
    # 4. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Shape of linked data after handling missing values: {linked_data.shape}")
    
    # 5. Determine if the trait and demographic features are biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 6. Validate the dataset and save cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_trait_biased,
        df=unbiased_linked_data,
        note="Dataset contains gene expression data from melanoma patients with information on long-term benefit from PD-1 based immunotherapy."
    )
    
    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Saved processed linked data to {out_data_file}")
    else:
        print("Dataset validation failed. Final linked data not saved.")
        
except Exception as e:
    print(f"Error in data linking process: {e}")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=None,
        df=None,
        note=f"Failed to link clinical and gene data. Error: {e}"
    )
    print("Dataset validation failed. Final linked data not saved.")